In [2]:
import json
try:
    import json
except:
    %pip install json
    import json

import os

try:
    from sortedcontainers import SortedDict
except:
    %pip install sortedcontainers
    from sortedcontainers import SortedDict


# Load data file - events_all.json

In [3]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start")
        year = start.split("-")[0]

        if year in number_ex_by_year:
            number_ex_by_year[year] += 1
        else:
            number_ex_by_year[year] = 1
        
    number_ex_by_year = SortedDict(number_ex_by_year)

   # print(number_ex_by_year)

# Create a forced network diagram

diagram with links between artists and organisations (via inclusion in exhibitions)

In [4]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    nodes = []
    links = []

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")

        nodes_ids  = [x['id'] for x in nodes]
       
        if org == "The ":
            continue
        if org == "MoMA PS1":
            continue
        if org not in nodes_ids:
            nodes.append({"id": org, "group": 1})

          
        influenced_by = event.get("influenced_by")
        for person in influenced_by:
            if person not in nodes_ids:
                nodes.append({"id": person, "group": 2})

            #links_sources  = [x['source'] for x in links]

            links.append({"source": org, "target": person, "value": 1})

    with open('data/summary/org_artist_links_nonmoma.json', 'w') as file:
        file.write(json.dumps({"nodes": nodes, "links": links}))
        

In [5]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    nodes = []
    links = []

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")

        nodes_ids  = [x['id'] for x in nodes]
       
        list_orgs = ['Tanager Gallery', "Brata Gallery", "Hansa Gallery", "MoMA PS1"]
        if org not in list_orgs:
            continue
        
        if org not in nodes_ids:
            nodes.append({"id": org, "group": 1})

        influenced_by = event.get("influenced_by")
        for person in influenced_by:
            if person not in nodes_ids:
                nodes.append({"id": person, "group": 2})

            links.append({"source": org, "target": person, "value": 1})


    list_links_persons  = [x['target'] for x in links]
    list_nodes_ids = [x['id'] for x in nodes]

    nonmoma_links = []
    moma_links = []

    for link in links:
        if link.get("source") not in ["MoMA PS1", "The Museum of Modern Art"]:
            nonmoma_links.append(link)
        else:
            moma_links.append(link)

    list_persons_nonmoma = [x['target'] for x in nonmoma_links]
    list_persons_moma = [x['target'] for x in moma_links]

    # get persons in moma list that are not in nonmoma list and then remove links including them 
    # and remove corresponding nodes

    list_persons_uniquetomoma= []
    for person in list_persons_moma:
        if person not in list_persons_nonmoma:
            list_persons_uniquetomoma.append(person)

    
    i = 0
    while i < len(links):
        link = links[i]
        if "target" in link: 
            person = link.get("target")
            if person in list_persons_uniquetomoma:
                
                del links[i]
        i += 1

    i = 0
 

    with open('data/summary/org_artist_links_10thstreet.json', 'w') as file:
        file.write(json.dumps({"nodes": nodes, "links": links}))


 
        